# Anti-CRISPR Exp15~Exp17: ProteinBERT + PSSM 融合

**功能**：本 notebook 总览与适用范围（PSSM 流水线 + 融合实验 + 基线/消融对照）。

**实验计划**（参考 `pssm融合后续实验计划_5b0b33fd.plan.md`）：
- **数据**：PSSM 由 UniRef50 + PSI-BLAST 生成；特征为 RPSSM(110) + PSSM-AC(200) + PSSM-composition(400) + DPC-PSSM(400)，对应 310/710/1110 三种维度。
- **实验组**：每个 seed 下依次跑 Ablation_RPSSM_310/710/1110（仅 PSSM）、Baseline_ProteinBERT（仅 BERT）、ProteinBERT_PSSM310/710/1110（融合），固定 seeds=[0,11,22,33,44]。
- **产出**：exp_df 明细、按 AUPRC 排序的 summary、以及 exp_results.csv / exp_summary.csv。

In [1]:
# ========== 依赖与路径 ==========
# 导入：numpy/pandas、sklearn 评估与划分、MLPClassifier/StandardScaler（PSSM-only 消融用）、
#       ProteinBERT 的 load_pretrained_model / FusionTrainConfig / run_finetune_with_pssm 等。
# 常量：PROJECT_ROOT, BENCHMARKS_DIR, WORK_ROOT；FEATURE_VARIANTS=['310','710','1110']；SEEDS=[0,11,22,33,44]。
import os
import numpy as np
import pandas as pd
from sklearn.metrics import average_precision_score, brier_score_loss, f1_score, matthews_corrcoef, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

from proteinbert import (
    OutputType,
    OutputSpec,
    FinetuningModelGenerator,
    load_pretrained_model,
    finetune,
    FusionTrainConfig,
    load_anticrispr_with_ids,
    load_feature_cache,
    attach_pssm_features,
    run_finetune_with_pssm,
)
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

PROJECT_ROOT = '/home/nemophila/projects/protein_bert'
BENCHMARKS_DIR = f'{PROJECT_ROOT}/anticrispr_benchmarks'
WORK_ROOT = f'{PROJECT_ROOT}/pssm_work'
FEATURE_VARIANTS = ['310', '710', '1110']
SEEDS = [0, 11, 22, 33, 44]


2026-02-13 15:52:36.470480: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


## 阶段0-1：PSSM 数据准备（命令级）

**说明**：以下步骤在命令行执行，用于生成 PSSM 文件与 310/710/1110 维特征缓存。若已跑过流水线且 `pssm_work/features/` 下已有 `pssm_features_*.csv`，可跳过本段，直接运行下方「加载数据」cell。

首次执行（数据库下载与建库）：

```bash
cd /home/nemophila/projects/protein_bert
conda install -c bioconda blast -y
bash scripts/pssm/00_download_uniref50.sh
mkdir -p /home/nemophila/projects/protein_bert/pssm_work/{fasta,pssm,logs,features}
```

生成 FASTA + 跑 PSI-BLAST + 提特征：

```bash
python scripts/pssm/00_prepare_fasta.py \
  --train-csv /home/nemophila/projects/protein_bert/anticrispr_benchmarks/anticrispr_binary.train.csv \
  --test-csv /home/nemophila/projects/protein_bert/anticrispr_benchmarks/anticrispr_binary.test.csv \
  --work-root /home/nemophila/projects/protein_bert/pssm_work

bash scripts/pssm/01_run_psiblast_batch.sh \
  /home/nemophila/projects/protein_bert/pssm_work/sample_manifest.csv \
  /home/nemophila/projects/protein_bert/blast_db/uniref50 8

bash scripts/pssm/02_retry_failed.sh \
  /home/nemophila/projects/protein_bert/pssm_work/sample_manifest.csv \
  /home/nemophila/projects/protein_bert/pssm_work/logs/failed_ids.txt \
  /home/nemophila/projects/protein_bert/blast_db/uniref50 4

python scripts/pssm/03_extract_rpssm_pssmac.py \
  --manifest-csv /home/nemophila/projects/protein_bert/pssm_work/sample_manifest.csv \
  --work-root /home/nemophila/projects/protein_bert/pssm_work

python scripts/pssm/04_build_feature_cache.py \
  --manifest-csv /home/nemophila/projects/protein_bert/pssm_work/sample_manifest.csv \
  --work-root /home/nemophila/projects/protein_bert/pssm_work \
  --variants 310,710,1110
```

In [2]:
# ========== 加载数据与特征 ==========
# 从 anticrispr_binary 读入 train/test，再按 FEATURE_VARIANTS 依次加载 pssm_features_310/710/1110（parquet 或 csv），
# 通过 attach_pssm_features 按 sample_id 对齐到训练/测试集，得到 train_df_310/710/1110、test_df_310/710/1110 及对应 feature_cols。
train_base_df, test_base_df = load_anticrispr_with_ids(BENCHMARKS_DIR, benchmark_name='anticrispr_binary')

variant_datasets = {}
for variant in FEATURE_VARIANTS:
    parquet_path = f'{WORK_ROOT}/features/pssm_features_{variant}.parquet'
    csv_path = f'{WORK_ROOT}/features/pssm_features_{variant}.csv'
    cache_path = parquet_path if os.path.exists(parquet_path) else csv_path
    feature_df, feature_cols = load_feature_cache(cache_path)

    train_df_v = attach_pssm_features(train_base_df, feature_df, feature_cols)
    test_df_v = attach_pssm_features(test_base_df, feature_df, feature_cols)
    variant_datasets[variant] = (train_df_v, test_df_v, feature_cols)

    print(f'variant={variant} train shape:', train_df_v.shape)
    print(f'variant={variant} test shape:', test_df_v.shape)
    print(f'variant={variant} feature dim:', len(feature_cols))

train_df_310, test_df_310, feature_cols_310 = variant_datasets['310']
train_df_710, test_df_710, feature_cols_710 = variant_datasets['710']
train_df_1110, test_df_1110, feature_cols_1110 = variant_datasets['1110']


variant=310 train shape: (1107, 313)
variant=310 test shape: (286, 313)
variant=310 feature dim: 310
variant=710 train shape: (1107, 713)
variant=710 test shape: (286, 713)
variant=710 feature dim: 710
variant=1110 train shape: (1107, 1113)
variant=1110 test shape: (286, 1113)
variant=1110 feature dim: 1110


In [3]:
# ========== 评估与阈值 ==========
# expected_calibration_error：ECE，按概率分 bin 算 |acc-conf| 加权和。
# evaluate_binary：给定阈值 thr，计算 AUC、AUPRC、F1、MCC、Brier、ECE。
# find_best_thr：在验证集上网格搜索使 F1 最大的阈值，供测试集评估使用。
def expected_calibration_error(y_true, y_prob, n_bins=10):
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    ids = np.digitize(y_prob, bins) - 1
    ece = 0.0
    n = len(y_true)
    for b in range(n_bins):
        m = ids == b
        if np.any(m):
            conf = float(np.mean(y_prob[m]))
            acc = float(np.mean(y_true[m]))
            ece += (np.sum(m) / n) * abs(acc - conf)
    return float(ece)

def evaluate_binary(y_true, y_prob, thr=0.5):
    y_cls = (y_prob >= thr).astype(int)
    return {
        'AUC': float(roc_auc_score(y_true, y_prob)),
        'AUPRC': float(average_precision_score(y_true, y_prob)),
        'F1': float(f1_score(y_true, y_cls)),
        'MCC': float(matthews_corrcoef(y_true, y_cls)),
        'Brier': float(brier_score_loss(y_true, y_prob)),
        'ECE': float(expected_calibration_error(y_true, y_prob, n_bins=10)),
        'Threshold': float(thr),
    }

def find_best_thr(y_true, y_prob):
    best_thr, best_f1 = 0.5, -1.0
    for thr in np.linspace(0.05, 0.95, 19):
        cur_f1 = f1_score(y_true, (y_prob >= thr).astype(int))
        if cur_f1 > best_f1:
            best_f1, best_thr = cur_f1, float(thr)
    return best_thr


In [4]:
# ========== 基线：仅 ProteinBERT ==========
# run_baseline_one_seed(seed)：用 train_df_310 的 seq/label 做 0.9/0.1 划分，加载预训练 BERT，
# 使用 get_model_with_hidden_layers_as_outputs 的 two_layer head 微调，验证集上 find_best_thr 后在测试集计算 AUC/AUPRC/F1/MCC/Brier/ECE。
# 不使用任何 PSSM 特征，作为 Baseline_ProteinBERT。
def run_baseline_one_seed(seed):
    sub_train, sub_valid = train_test_split(
        train_df_310[['seq', 'label']], test_size=0.1, stratify=train_df_310['label'], random_state=seed
    )

    output_type = OutputType(False, 'binary')
    output_spec = OutputSpec(output_type, [0, 1])

    pretrained_model_generator, input_encoder = load_pretrained_model(
        local_model_dump_dir=f'{PROJECT_ROOT}/proteinbert_models',
        download_model_dump_if_not_exists=True,
        validate_downloading=False,
    )

    mg = FinetuningModelGenerator(
        pretrained_model_generator,
        output_spec=output_spec,
        pretraining_model_manipulation_function=get_model_with_hidden_layers_as_outputs,
        dropout_rate=0.4,
        head_type='two_layer',
        loss_type='bce',
        lr=2e-5,
    )

    finetune(
        mg,
        input_encoder,
        output_spec,
        sub_train['seq'],
        sub_train['label'],
        sub_valid['seq'],
        sub_valid['label'],
        seq_len=512,
        batch_size=8,
        max_epochs_per_stage=8,
        begin_with_frozen_pretrained_layers=True,
        n_final_epochs=0,
    )

    model = mg.create_model(512)
    X_valid = input_encoder.encode_X(sub_valid['seq'].tolist(), 512)
    valid_prob = model.predict(X_valid, batch_size=8, verbose=0).reshape(-1)
    thr = find_best_thr(sub_valid['label'].to_numpy(), valid_prob)

    X_test = input_encoder.encode_X(test_df_310['seq'].tolist(), 512)
    test_prob = model.predict(X_test, batch_size=8, verbose=0).reshape(-1)
    return evaluate_binary(test_df_310['label'].to_numpy(), test_prob, thr=thr)


In [5]:
# ========== 主实验：单 cell 跑齐所有实验 ==========
# run_pssm_only_one_seed：仅用 PSSM 特征（StandardScaler + MLP 128->64），同 seed 下 train/valid 划分、找阈值、测试集评估。
# 对每个 seed 依次执行 7 组并写入 all_rows：
#   1) Ablation_RPSSM_310  2) Ablation_RPSSM_710  3) Ablation_RPSSM_1110
#   4) Baseline_ProteinBERT  5) ProteinBERT_PSSM310  6) ProteinBERT_PSSM710  7) ProteinBERT_PSSM1110
# 融合实验使用 FusionTrainConfig 与 run_finetune_with_pssm，每次重新 load_pretrained_model 以保证独立权重。
# 最后 exp_df = pd.DataFrame(all_rows)。
def run_pssm_only_one_seed(train_df, test_df, feature_cols, seed):
    sub_train, sub_valid = train_test_split(
        train_df, test_size=0.1, stratify=train_df['label'], random_state=seed,
    )
    X_tr = sub_train[feature_cols].to_numpy(dtype=np.float32)
    X_va = sub_valid[feature_cols].to_numpy(dtype=np.float32)
    X_te = test_df[feature_cols].to_numpy(dtype=np.float32)
    y_tr = sub_train['label'].astype(int).to_numpy()
    y_va = sub_valid['label'].astype(int).to_numpy()
    y_te = test_df['label'].astype(int).to_numpy()
    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_tr)
    X_va = scaler.transform(X_va)
    X_te = scaler.transform(X_te)
    clf = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', max_iter=200,
                        random_state=seed, early_stopping=True, validation_fraction=0.1)
    clf.fit(X_tr, y_tr)
    valid_prob = clf.predict_proba(X_va)[:, 1]
    thr = find_best_thr(y_va, valid_prob)
    test_prob = clf.predict_proba(X_te)[:, 1]
    return evaluate_binary(y_te, test_prob, thr=thr)

cfg = FusionTrainConfig(
    seq_len=512, batch_size=8, frozen_epochs=6, unfrozen_epochs=12,
    frozen_lr=1e-4, unfrozen_lr=2e-5, pssm_dropout=0.3, global_dropout=0.3,
    pssm_hidden_dim=128, global_hidden_dim=128, global_bottleneck_dim=64,
    fusion_hidden_dim=128, use_hidden_global_concat=True,
)

all_rows = []
for seed in SEEDS:
    print(f'--- seed={seed} ---')

    m = run_pssm_only_one_seed(train_df_310, test_df_310, feature_cols_310, seed)
    all_rows.append({'Exp': 'Ablation_RPSSM_310', 'Seed': seed, **m})

    m = run_pssm_only_one_seed(train_df_710, test_df_710, feature_cols_710, seed)
    all_rows.append({'Exp': 'Ablation_RPSSM_710', 'Seed': seed, **m})

    m = run_pssm_only_one_seed(train_df_1110, test_df_1110, feature_cols_1110, seed)
    all_rows.append({'Exp': 'Ablation_RPSSM_1110', 'Seed': seed, **m})

    base_metrics = run_baseline_one_seed(seed)
    all_rows.append({'Exp': 'Baseline_ProteinBERT', 'Seed': seed, **base_metrics})

    pmg, enc = load_pretrained_model(local_model_dump_dir=f'{PROJECT_ROOT}/proteinbert_models',
        download_model_dump_if_not_exists=True, validate_downloading=False)
    m = run_finetune_with_pssm(pmg, enc, train_df_310, test_df_310, feature_cols_310, seed=seed, cfg=cfg)
    all_rows.append({'Exp': 'ProteinBERT_PSSM310', 'Seed': seed, **m})

    pmg, enc = load_pretrained_model(local_model_dump_dir=f'{PROJECT_ROOT}/proteinbert_models',
        download_model_dump_if_not_exists=True, validate_downloading=False)
    m = run_finetune_with_pssm(pmg, enc, train_df_710, test_df_710, feature_cols_710, seed=seed, cfg=cfg)
    all_rows.append({'Exp': 'ProteinBERT_PSSM710', 'Seed': seed, **m})

    pmg, enc = load_pretrained_model(local_model_dump_dir=f'{PROJECT_ROOT}/proteinbert_models',
        download_model_dump_if_not_exists=True, validate_downloading=False)
    m = run_finetune_with_pssm(pmg, enc, train_df_1110, test_df_1110, feature_cols_1110, seed=seed, cfg=cfg)
    all_rows.append({'Exp': 'ProteinBERT_PSSM1110', 'Seed': seed, **m})

exp_df = pd.DataFrame(all_rows)
exp_df

--- seed=0 ---
[2026_02_13-15:53:17] Training set: Filtered out 0 of 996 (0.0%) records of lengths exceeding 510.
[2026_02_13-15:53:17] Validation set: Filtered out 0 of 111 (0.0%) records of lengths exceeding 510.
[2026_02_13-15:53:17] Training with frozen pretrained layers...


2026-02-13 15:53:17.246819: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2026-02-13 15:53:17.247833: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2026-02-13 15:53:17.297053: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2a:00.0 name: NVIDIA L40S computeCapability: 8.9
coreClock: 2.52GHz coreCount: 142 deviceMemorySize: 44.53GiB deviceMemoryBandwidth: 804.75GiB/s
2026-02-13 15:53:17.297320: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:ab:00.0 name: NVIDIA L40S computeCapability: 8.9
coreClock: 2.52GHz coreCount: 142 deviceMemorySize: 44.53GiB deviceMemoryBandwidth: 804.75GiB/s
2026-02-13 15:53:17.297363: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2026-02-13 15:53:17.2

Epoch 1/8


2026-02-13 15:53:28.177470: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2026-02-13 15:53:29.077127: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2026-02-13 15:53:29.097956: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2026-02-13 15:53:29.098491: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2026-02-13 15:53:31.874561: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2026-02-13 15:53:32.181194: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


125/125 [==============================] - 46s 203ms/step - loss: 0.6380 - val_loss: 0.4520
Epoch 2/8
125/125 [==============================] - 2s 15ms/step - loss: 0.4182 - val_loss: 0.3837
Epoch 3/8
125/125 [==============================] - 2s 14ms/step - loss: 0.3365 - val_loss: 0.3625
Epoch 4/8
125/125 [==============================] - 2s 14ms/step - loss: 0.3848 - val_loss: 0.3485
Epoch 5/8
125/125 [==============================] - 2s 15ms/step - loss: 0.3304 - val_loss: 0.3459
Epoch 6/8
125/125 [==============================] - 2s 14ms/step - loss: 0.2960 - val_loss: 0.3422
Epoch 7/8
125/125 [==============================] - 2s 14ms/step - loss: 0.2711 - val_loss: 0.3361
Epoch 8/8
125/125 [==============================] - 2s 14ms/step - loss: 0.2550 - val_loss: 0.3477
[2026_02_13-15:54:20] Training the entire fine-tuned model...
[2026_02_13-15:54:56] Incompatible number of optimizer weights - will not initialize them.
Epoch 1/8
125/125 [==============================] - 47

,Exp,Seed,AUC,AUPRC,F1,MCC,Brier,ECE,Threshold
0,Ablation_RPSSM_310,0,0.887870,0.494901,0.357143,0.288675,0.070647,0.074914,0.45
1,Ablation_RPSSM_710,0,0.873817,0.508017,0.461538,0.405966,0.064152,0.043911,0.30
2,Ablation_RPSSM_1110,0,0.901331,0.530879,0.481928,0.451162,0.073010,0.074751,0.35
3,Baseline_ProteinBERT,0,0.902367,0.637022,0.541667,0.502079,0.061589,0.058164,0.55
4,ProteinBERT_PSSM310,0,0.882396,0.577853,0.461538,0.456505,0.061487,0.042305,0.80
5,ProteinBERT_PSSM710,0,0.931213,0.693828,0.592593,0.550646,0.056423,0.038251,0.55
6,ProteinBERT_PSSM1110,0,0.939793,0.710081,0.590164,0.549942,0.053087,0.045446,0.40
7,Ablation_RPSSM_310,11,0.846154,0.470211,0.412698,0.349225,0.077098,0.045828,0.35
8,Ablation_RPSSM_710,11,0.892308,0.485872,0.468750,0.413701,0.070136,0.039131,0.25
9,Ablation_RPSSM_1110,11,0.902959,0.589331,0.461538,0.407692,0.059286,0.051762,0.45


In [6]:
# ========== 汇总表 ==========
# 按 Exp 分组，对 AUC/AUPRC/F1/MCC/Brier/ECE 求 mean 与 std，再按 AUPRC mean 降序排列，便于对比各实验组。
summary = exp_df.groupby('Exp')[['AUC','AUPRC','F1','MCC','Brier','ECE']].agg(['mean','std'])
summary = summary.sort_values(('AUPRC', 'mean'), ascending=False)
summary


AUC               AUPRC                  F1  \
                          mean       std      mean       std      mean   
Exp                                                                      
ProteinBERT_PSSM1110  0.932751  0.013180  0.693407  0.038006  0.578494   
ProteinBERT_PSSM710   0.910917  0.017413  0.646984  0.042650  0.573196   
ProteinBERT_PSSM310   0.901331  0.028340  0.641922  0.060256  0.506556   
Baseline_ProteinBERT  0.888047  0.022444  0.615958  0.027875  0.468750   
Ablation_RPSSM_1110   0.910888  0.011134  0.572590  0.037593  0.509068   
Ablation_RPSSM_710    0.888314  0.015644  0.519801  0.037194  0.467661   
Ablation_RPSSM_310    0.878935  0.019749  0.493284  0.013242  0.394280   

                                     MCC               Brier            \
                           std      mean       std      mean       std   
Exp                                                                      
ProteinBERT_PSSM1110  0.043616  0.538457  0.045809  0.058557  0.009281   
ProteinBERT_PSSM710   0.053450  0.534978  0.059166  0.055136  0.003485   
ProteinBERT_PSSM310   0.075690  0.475807  0.068423  0.055323  0.006938   
Baseline_ProteinBERT  0.069330  0.429053  0.065064  0.058701  0.003786   
Ablation_RPSSM_1110   0.037329  0.463947  0.037736  0.061633  0.006944   
Ablation_RPSSM_710    0.013396  0.419751  0.027350  0.066863  0.005284   
Ablation_RPSSM_310    0.050467  0.333280  0.052844  0.070957  0.006008   

                           ECE            
                          mean       std  
Exp                                       
ProteinBERT_PSSM1110  0.056047  0.019567  
ProteinBERT_PSSM710   0.039283  0.007325  
ProteinBERT_PSSM310   0.042860  0.007589  
Baseline_ProteinBERT  0.045868  0.010042  
Ablation_RPSSM_1110   0.051236  0.014004  
Ablation_RPSSM_710    0.042576  0.008706  
Ablation_RPSSM_310    0.058589  0.014761

In [7]:
# ========== 保存结果 ==========
# 将 exp_df 写入 pssm_work/features/exp_results.csv（每行：Exp, Seed, AUC, AUPRC, F1, MCC, Brier, ECE, Threshold），
# 将 summary 写入 exp_summary.csv，供后续报告或 06_generate_plan_reports 使用。
os.makedirs(f'{WORK_ROOT}/features', exist_ok=True)
report_path = f'{WORK_ROOT}/features/exp_results.csv'
summary_path = f'{WORK_ROOT}/features/exp_summary.csv'
exp_df.to_csv(report_path, index=False)
summary.to_csv(summary_path)
print('saved:', report_path)
print('saved:', summary_path)


saved: /home/nemophila/projects/protein_bert/pssm_work/features/exp_results.csv
saved: /home/nemophila/projects/protein_bert/pssm_work/features/exp_summary.csv
